In [2]:
import numpy as np;
import torch

input = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]], dtype='float32')
# print(input)
# print(input.shape)
target = np.array([[56],
                   [81],
                   [119],
                   [22],
                   [103]], dtype='float32')
# print(target)

# Convert inputs and targets to tensors

inputs_t = torch.from_numpy(input)
targets_t = torch.from_numpy(target)

print(inputs_t)
print(targets_t)
     

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


linear regression from scratch

In [4]:
Weights = torch.randn(3, 1, requires_grad=True)
biases = torch.randn(1, requires_grad=True)
print(Weights)
print(biases)

tensor([[ 0.6368],
        [ 0.4286],
        [-0.8074]], requires_grad=True)
tensor([-1.6966], requires_grad=True)


model

In [5]:
def model(x):
    return x @ Weights + biases

pred = model(inputs_t)
pred

tensor([[38.7860],
        [42.2928],
        [64.3077],
        [51.8097],
        [26.8683]], grad_fn=<AddBackward0>)

mse loss

In [6]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

loss = mse(pred, targets_t)
print(loss)


tensor(2294.0945, grad_fn=<DivBackward0>)


compute gradiant

In [7]:
loss.backward()

print(Weights)
print(Weights.grad)

print(biases)
print(biases.grad)

Weights.grad.zero_()
biases.grad.zero_()
print(Weights.grad)
print(biases.grad)

tensor([[ 0.6368],
        [ 0.4286],
        [-0.8074]], requires_grad=True)
tensor([[-4699.8818],
        [-7166.0659],
        [-4246.3506]])
tensor([-1.6966], requires_grad=True)
tensor([-62.7742])
tensor([[0.],
        [0.],
        [0.]])
tensor([0.])


adjust weight and bias

In [8]:
pred = model(inputs_t)
print(pred)

print(Weights.grad)
print(biases.grad)

with torch.no_grad():
    Weights -= Weights.grad * 1e-5
    biases -= biases.grad * 1e-5
    Weights.grad.zero_()
    biases.grad.zero_()

print(Weights)
print(biases)

pred = model(inputs_t)
loss = mse(pred, targets_t)
print(loss)

tensor([[38.7860],
        [42.2928],
        [64.3077],
        [51.8097],
        [26.8683]], grad_fn=<AddBackward0>)
tensor([[0.],
        [0.],
        [0.]])
tensor([0.])
tensor([[ 0.6368],
        [ 0.4286],
        [-0.8074]], requires_grad=True)
tensor([-1.6966], requires_grad=True)
tensor(2294.0945, grad_fn=<DivBackward0>)


train the model

In [9]:
for i in range(160):
    pred = model(inputs_t)
    loss = mse(pred, targets_t)
    loss.backward()
    with torch.no_grad():
        Weights -= Weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        Weights.grad.zero_()
        biases.grad.zero_()

pred = model(inputs_t)
loss = mse(pred, targets_t)
print(loss)

pred

targets_t

tensor(121.8993, grad_fn=<DivBackward0>)


tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

using inbuild

In [10]:
import torch.nn as nn
import torch
import numpy as np

inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37],[69, 96, 70],
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69,96, 70],
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96,70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119],
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119],
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]],dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

inputs

targets

from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(inputs, targets)
train_ds[:]

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

for batch, (xb, yb) in enumerate(train_dl):
    print(f'batch: {batch+1}')
    print(xb)
    print(yb)
    print('='*40)

model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

preds = model(inputs)
preds

# loss

import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

# Calculate loss
loss = loss_fn(model(inputs), targets)
print(loss)

# optimizer 
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {round(loss.item(), 3)} ,{xb} , {yb}')

fit(250, model, loss_fn, opt, train_dl)

preds = model(inputs)
preds

targets

from sklearn.metrics import mean_squared_error
print(mean_squared_error(targets.detach().numpy(),preds.detach().numpy()))

batch: 1
tensor([[102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.]])
tensor([[ 22.,  37.],
        [ 22.,  37.],
        [ 81., 101.],
        [119., 133.],
        [ 81., 101.]])
batch: 2
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [119., 133.],
        [103., 119.],
        [103., 119.]])
batch: 3
tensor([[ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.]])
tensor([[103., 119.],
        [ 56.,  70.],
        [119., 133.],
        [ 56.,  70.],
        [ 56.,  70.]])
Parameter containing:
tensor([[-0.0872, -0.1310, -0.1949],
        [ 0.2335, -0.3889, -0.3542]], requires_grad=True)
Parameter containing:
tensor([-0.0003,  0.0243], requires_grad=True)
tensor(15440.7871, grad_fn=<Mse

# Exercise

In [11]:
# 1
fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 10.406 ,tensor([[ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.]]) , tensor([[ 56.,  70.],
        [119., 133.],
        [103., 119.],
        [ 56.,  70.],
        [ 22.,  37.]])
Epoch [20/1000], Loss: 10.611 ,tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.]]) , tensor([[ 22.,  37.],
        [ 81., 101.],
        [103., 119.],
        [ 22.,  37.],
        [119., 133.]])
Epoch [30/1000], Loss: 9.811 ,tensor([[ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.]]) , tensor([[ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [ 81., 101.],
        [ 56.,  70.]])
Epoch [40/1000], Loss: 10.059 ,tensor([[ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
   

In [12]:
# 2
model2 = nn.Linear(3,2)
print(model2.weight)
print(model2.bias)
opt2 = torch.optim.SGD(model2.parameters(), lr=0.1)
fit(250, model2, loss_fn, opt2, train_dl)
preds = model2(inputs)
print(preds)
print(targets)

Parameter containing:
tensor([[ 0.3535,  0.3796, -0.5671],
        [ 0.3741, -0.2101, -0.5059]], requires_grad=True)
Parameter containing:
tensor([0.4446, 0.1313], requires_grad=True)
Epoch [10/250], Loss: nan ,tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]]) , tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [ 81., 101.]])
Epoch [20/250], Loss: nan ,tensor([[102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.]]) , tensor([[ 22.,  37.],
        [ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [103., 119.]])
Epoch [30/250], Loss: nan ,tensor([[ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.]]) , tensor([[ 81., 101.],
        [ 56.,  70.],
        [ 22.,  37.],
        [119., 133

In [13]:
# 3
model3 = nn.Linear(3,2)
opt3 = torch.optim.SGD(model3.parameters(), lr=0.0000001)
fit(250, model3, loss_fn, opt3, train_dl)
preds = model3(inputs)
print(preds)
print(targets)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(targets.detach().numpy(),preds.detach().numpy()))

Epoch [10/250], Loss: 7234.905 ,tensor([[ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]]) , tensor([[ 81., 101.],
        [ 22.,  37.],
        [ 22.,  37.],
        [103., 119.],
        [119., 133.]])
Epoch [20/250], Loss: 7947.881 ,tensor([[ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.]]) , tensor([[ 56.,  70.],
        [103., 119.],
        [ 22.,  37.],
        [119., 133.],
        [103., 119.]])
Epoch [30/250], Loss: 6066.879 ,tensor([[102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]]) , tensor([[ 22.,  37.],
        [103., 119.],
        [119., 133.],
        [ 22.,  37.],
        [ 81., 101.]])
Epoch [40/250], Loss: 3853.214 ,tensor([[102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.]

In [14]:
preds = model(torch.FloatTensor([70,34,45]))
preds

tensor([31.5671, 46.4995], grad_fn=<AddBackward0>)